## Migrating OpenAI Function calling (Tool Use) to Converse API Function Calling with Amazon Bedrock Converse API

In this example notebook, we'll explore how to migrate OpenAI Function calling to work with function calling provided by the Converse API for Amazon Bedrock.

In [1]:
## Ensure you have at least version 1.34.117 of boto3
# !pip3 install -qU boto3

We can now import the relevant libraries.

In [14]:
import boto3
from typing import Dict, List 
import pprint 

print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.34.117


Let's also define a few variables. Make sure to adjust these parameters according to your needs.

First let's define a set of Bedrock models we'll test with Function Calling.

In [3]:
MODEL_IDS = [
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0',
    'mistral.mistral-large-2402-v1:0',
    'cohere.command-r-plus-v1:0',
    'cohere.command-r-v1:0'
]

Now we'll set up our boto3 client.

In [4]:
region = 'us-east-1'
print('Using region: ', region)

bedrock = boto3.client(
    service_name = 'bedrock-runtime',
    region_name = region,
    )

Using region:  us-east-1


#### Define a utility function to migrate an existing OpenAI function call to a Bedrock Converse call.

In [5]:
def oai_call_to_bedrock_call(oai_call: Dict) -> Dict:
    """
    Converts an OpenAI function call to a Bedrock Converse call.

    Args:
        oai_call (Dict): The OAI function call to be converted.

    Returns:
        Dict: The converted Bedrock Converse call.
    """
    _functions = []

    # Each function in OpenAI becomes a "toolSpec" for Converse. The OpenAI function
    # parameters become a Converse "inputSchema".

    for _function in oai_call['functions']: 
        _functions.append({
            "toolSpec": {
                "name": _function["name"],
                "description": _function["description"],
                "inputSchema": {
                    "json": _function["parameters"]
                }
            }
        })

    _messages = []

    # Migrate the message "content" to have a "text" field.

    for _message in oai_call['messages']:
        _messages.append({
            "role": _message["role"],
            "content": [ 
                {
                    "text": _message["content"]
                }                
            ]})
    
    # Lastly, wrap the tool definitions in a "toolConfig" field.

    return {
        "messages": _messages,
        "toolConfig": {
            # Optionally add '"toolChoice": {"auto": {}},'
            "tools": _functions
        }
    } 

#### Define a utility function for exercising Bedrock Converse to have it select the right tool and parameters

In [15]:
def converse_with_tools(messages: List[Dict], system: str="", toolConfig: Dict={},
                        modelId = 'anthropic.claude-3-sonnet-20240229-v1:0',
                        temperature: float=0.0,
                        topP: float=1.0,
                        maxTokens: int=512) -> str:
    """Utility function to call the Bedrock Converse API and get a function calling response from the model.

    Args:
        messages (List[Dict]): messages to pass to the model
        system (str, optional): system prompt
        toolConfig (Dict, optional): configuration of tools for the model to choose from
        modelId (str, optional): which Bedrock model to use 

    Returns:
        str: response from the model
    """
    system_prompt = [{"text": system}]

    response = bedrock.converse(
        modelId=modelId,
        system=system_prompt,
        messages=messages,
        toolConfig=toolConfig,
        inferenceConfig={
            'temperature': temperature,
            'topP': topP,
            'maxTokens': maxTokens
            }
    )
    return response

#### Define a few simle example OpenAI function calls

In [16]:
open_ai_meeting_call = {
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {
      "role": "user",
      "content": "Schedule a meeting with John Doe next Tuesday at 3 PM."
    }
  ],
  "functions": [
    {
      "name": "schedule_meeting",
      "description": "Please schedule a meeting.",
      "parameters": {
        "type": "object",
        "properties": {
          "attendee": {
            "type": "string",
            "description": "Attendee for the meeting"
          },
          "date": {
            "type": "string",
            "description": "Date of the meeting"
          },
          "time": {
            "type": "string",
            "description": "Time of the meeting"
          }
        }
      }
    }
  ]
}

In [17]:
open_ai_stock_call = {
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {
      "role": "user",
      "content": "What's the current price of Apple stocks?"
    }
  ],
  "functions": [
    {
      "name": "get_stock_price",
      "description": "Get current stock price",
      "parameters": {
        "type": "object",
        "properties": {
          "ticker_symbol": {
            "type": "string",
            "description": "Ticker symbol of the stock"
          }
        }
      }
    }
  ]
}

In [18]:
open_ai_travel_call = {
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {
      "role": "user",
      "content": "I need to book a trip from Bonn to Amsterdam for my wife, mother and by two sons and daughter. I will also be joining them. The airline must fly direct."
    }
  ],
  "functions": [
    {
      "name": "book_travel",
      "description": "Book travel",
      "parameters": {
        "type": "object",
        "properties": {
          "destination": {
            "type": "string",
            "description": "Your travel destination."
          },
          "departure": {
            "type": "string",
            "description": "From where are you traveling"
          },
          "number_people": {
            "type": "string",
            "description": "How many people are traveling"
          },
          "travel_mode": {
            "type": "string",
            "description": "What mode of travel will it be."
          }
        }
      }
    }
  ]
}

#### Show a sample converted function call

In [19]:
oai_call_to_bedrock_call(open_ai_meeting_call)

{'messages': [{'role': 'user',
   'content': [{'text': 'Schedule a meeting with John Doe next Tuesday at 3 PM.'}]}],
 'toolConfig': {'tools': [{'toolSpec': {'name': 'schedule_meeting',
     'description': 'Please schedule a meeting.',
     'inputSchema': {'json': {'type': 'object',
       'properties': {'attendee': {'type': 'string',
         'description': 'Attendee for the meeting'},
        'date': {'type': 'string', 'description': 'Date of the meeting'},
        'time': {'type': 'string',
         'description': 'Time of the meeting'}}}}}}]}}

#### Show the use of each of these example function calls with Bedrock Converse against a set of Bedrock models

In [22]:
calls = [open_ai_meeting_call, open_ai_stock_call, open_ai_travel_call]
for call in calls:
    print(f"Example call: {call['messages'][0]['content']}\n")
    bedrock_call = oai_call_to_bedrock_call(call) 

    for model_id in MODEL_IDS:
        print(f"======== Performing function calling with model: {model_id} ========\n")
        output = converse_with_tools(bedrock_call['messages'], 
                                system="you are a helpful assistant.",
                                modelId=model_id,
                                toolConfig=bedrock_call['toolConfig'])
        pprint.pprint(output['output'])
        print("\n")


Example call: Schedule a meeting with John Doe next Tuesday at 3 PM.

======== Performing function calling with model: anthropic.claude-3-sonnet-20240229-v1:0 ========

{'message': {'content': [{'text': 'Here is how to schedule the meeting using '
                                  'the available tool:'},
                         {'toolUse': {'input': {'attendee': 'John Doe',
                                                'date': 'Next Tuesday',
                                                'time': '3 PM'},
                                      'name': 'schedule_meeting',
                                      'toolUseId': 'tooluse_KeFQRzThQTOow-1l4mbpgA'}}],
             'role': 'assistant'}}


======== Performing function calling with model: anthropic.claude-3-haiku-20240307-v1:0 ========

{'message': {'content': [{'toolUse': {'input': {'attendee': 'John Doe',
                                                'date': 'next Tuesday',
                                                'tim